# Read FHX file

In [45]:
import pandas as pd
import re
import numpy as np

In [46]:
PATH = r"/Users/Vesal/Desktop/AltaML/Project/Use Case 1/Data/OneDrive_1_1-13-2021/"#AltaML_Sample_FHX_SAMA_Data/"
# FILE_NAME = 'AltaML_FHX_Sample.txt'
FILE_NAME = 'Reduced_FHX_file.txt'

data = open(PATH + FILE_NAME, 'r').read().strip()

# Define Module Extraction Functions

## 1- MODULE_CLASS

In [47]:
def module_class_variables(data):
    
    module_class = data.split('\nMODULE_CLASS')[1:]
    
    module_class_lists = []
    actual_class_module = []
    
    if len(module_class) > 0:
        module_class = ['MODULE_CLASS' + modules for modules in  module_class]
        
        for module in module_class:

            regex=re.compile(r'NAME=.* CATEGORY')
            module_name = regex.findall(module)[0][6:-10]
            
            module_class_lists.append([module_name])
            actual_class_module.append(module)

        return actual_class_module, module_class_lists
    
    else:
        return [[np.nan]]

## 2- MODULE_INSTANCE

In [48]:
def module_instance_variables(data):
    
    instance_modul_1 = data.split('MODULE_INSTANCE')[1:]
    
    module_instance_lists = []
    actual_instance_module = []
    
    if len(instance_modul_1) > 0:
        instance_modul_1 = ['MODULE_INSTANCE' + inst_modu for inst_modu in  instance_modul_1]
        
        for inst in instance_modul_1:

            # tag:
            regex=re.compile(r'TAG=.* PLANT_AREA')
            tag = regex.findall(inst)[0][5:-12]
            
            module_instance_lists.append([tag])
            actual_instance_module.append(inst)


        return actual_instance_module, module_instance_lists
    
    else:
        return [[np.nan]]

## 3- MODULE (INDEPENDENT)

In [49]:
def independent_module_variables(data):
    
    independent_modules = data.split('MODULE TAG')[1:]
    
    independent_modul_lists = []
    actual_independent_module = []
    
    if len(independent_modules) > 0:
        independent_modules = ['MODULE TAG' + ind_modu for ind_modu in  independent_modules]
        
        for ind in independent_modules:

            # tag:
            regex=re.compile(r'TAG=.* PLANT_AREA')
            tag = regex.findall(ind)[0][5:-12]
            
            independent_modul_lists.append([tag])
            actual_independent_module.append(ind)


        return actual_independent_module, independent_modul_lists
    
    else:
        return [[np.nan]]

# Create module dataframe

## 1- Class module

In [50]:
actual_class_module, class_module_name = module_class_variables(data)
class_module = pd.DataFrame(class_module_name, columns=['module_name'])
class_module['module_type'] = 'MODULE_CLASS'
class_module

,module_name,module_type
0,KS_AI_SIS_HH,MODULE_CLASS
1,KS_AI_STD,MODULE_CLASS


## 2- Instance module

In [51]:
actual_inst_module, inst_module_name = module_instance_variables(data)
instance_module = pd.DataFrame(inst_module_name, columns=['module_name'])
instance_module['module_type'] = 'MODULE_INSTANCE'
instance_module

,module_name,module_type
0,TI_580920,MODULE_INSTANCE
1,PI_580911B,MODULE_INSTANCE


## 3- Independent module

In [52]:
actual_ind_module, ind_module_name = independent_module_variables(data)
independent_modules = pd.DataFrame(ind_module_name, columns=['module_name'])
independent_modules['module_type'] = 'MODULE'
independent_modules

,module_name,module_type
0,FI_580931,MODULE
1,FIC_580931,MODULE
2,FI_580921,MODULE
3,FIC_580921,MODULE
4,PY-580959,MODULE


# Combine all modules and select the desired ones

In [53]:
all_modules = independent_modules.append(class_module).append(instance_module).reset_index(drop=True)
all_modules 

,module_name,module_type
0,FI_580931,MODULE
1,FIC_580931,MODULE
2,FI_580921,MODULE
3,FIC_580921,MODULE
4,PY-580959,MODULE
5,KS_AI_SIS_HH,MODULE_CLASS
6,KS_AI_STD,MODULE_CLASS
7,TI_580920,MODULE_INSTANCE
8,PI_580911B,MODULE_INSTANCE


In [54]:
all_actual_modules = actual_ind_module + actual_class_module + actual_inst_module

In [55]:
selected_modules_id = [0,1,2,3,4,5,6,7,8]

# Create function blocks dataframes from selected modules

## 1- Define function_block extractor function

In [56]:
def function_block_variables(module, module_name):

    func_modul_1 = module.split('FUNCTION_BLOCK')[1:]
    
    func_lists = []
    
    if len(func_modul_1) > 0:
    
        func_modul_1 = ['FUNCTION_BLOCK' + func for func in  func_modul_1]
        
        for func in func_modul_1:

            # name:
            regex=re.compile(r'NAME=.* DEFINITION')
            func_name = regex.findall(func)[0][6:-12]

            # definition
            regex=re.compile(r'DEFINITION=.*')
            func_def = regex.findall(func)[0][12:-1]
            
            func_lists.append([module_name, func_name, func_def])

        return func_lists
    
    else:
        return [[module_name, np.nan, np.nan]]    

## 2- Find function blocks for each selected module

In [57]:
modules = data.split('\nMODULE_CLASS')[1:]
modules = ['MODULE_CLASS' + module for module in modules]

module_function_block_list = []
for module_id in selected_modules_id:
    module_function_block_list.append(function_block_variables(all_actual_modules[module_id], 
                                            all_modules.iloc[module_id]['module_name']))
    
module_function_block_list = [item for sublist in module_function_block_list for item in sublist]

In [58]:
function_blocks = pd.DataFrame(module_function_block_list, columns=['module_name', 'function_block_name', 
                                                  'functon_block_definition'])

function_blocks

,module_name,function_block_name,functon_block_definition
0,FI_580931,AI2,AI
1,FI_580931,PV-COMPARE,CALC
2,FI_580931,HI_HI_ALARM,CND
3,FI_580931,HI_ALARM,CND
4,FI_580931,LO_ALARM,CND
...,...,...,...
260,PI_580911B,HI_HI_ALARM,CND
261,PI_580911B,HI_ALARM,CND
262,PI_580911B,LO_ALARM,CND
263,PI_580911B,LO_LO_ALARM,CND


In [59]:
function_blocks['functon_block_definition'].value_counts()

CND                      106
CALC                      28
SGCR                      21
ACT                       18
PDE                       12
ADD                        9
PIDWCALARM                 6
RTLM                       6
NOT                        6
AO                         6
AI                         6
AGA_US                     6
AND                        6
__5EDD6135_11352FAD__      3
__5DF27C84_007821F1__      3
__5ED68511_2D968C5E__      3
AIWCALARM                  3
SCLR                       3
__5EDD5F76_112E5D7E__      3
__5EB04632_00A3E88B__      3
BG                         3
CROSS_LIMIT                3
__5ED0355E_2784274A__      1
Name: functon_block_definition, dtype: int64

# Create attribute instance dataframes from selected modules

## 1- Define instance attribute extractor function

In [60]:
def attribute_instance_variables(module, module_name):

    attr_inst_modul_1 = module.split('ATTRIBUTE_INSTANCE')[1:]
    
    if len(attr_inst_modul_1) > 0:
        attr_inst_modul_1 = ['ATTRIBUTE_INSTANCE' + attr for attr in  attr_inst_modul_1]
        
        attr_instance_lists = []
        
        for attr_inst in attr_inst_modul_1:
            
            # attribute instant name
            regex=re.compile(r'NAME=.*')
            attr_instance_name = regex.findall(attr_inst)[0][6:-1]
            
            try:
                regex=re.compile(r'VALUE { REF=".*" }')
                attr_ref = regex.findall(attr_inst)[0][13:-3]
            except:
                attr_ref = np.nan
            
            attr_instance_lists.append([module_name, attr_instance_name, attr_ref])

        return attr_instance_lists

    else:
        return [[module_name, np.nan]]

## 2- Find instance attributes for each selected module

In [61]:
modules = data.split('\nMODULE_CLASS')[1:]
modules = ['MODULE_CLASS' + module for module in modules]

attr_inst_list = []
for module_id in selected_modules_id:
    attr_inst_list.append(attribute_instance_variables(all_actual_modules[module_id], 
                                            all_modules.iloc[module_id]['module_name']))
    
attr_inst_list = [item for sublist in attr_inst_list for item in sublist]

In [62]:
instance_attribute = pd.DataFrame(attr_inst_list, columns=['module_name', 'attribute_instance_name', 
                                                          'attr_instance_ref']).replace(
                                                '', np.nan).dropna().reset_index(drop=True)
instance_attribute
# instance_attribute

,module_name,attribute_instance_name,attr_instance_ref
0,FI_580931,TEMP_IN,//TI_580920/AI1/OUT
1,FI_580931,PRESS_IN,//AUX_BMS_IO/PIT911B/OUT
2,FIC_580931,IN,//FI_580931/AI1/OUT
3,FI_580921,TEMP_IN,//TI_580920/AI1/OUT
4,FI_580921,PRESS_IN,//AUX_BMS_IO/PIT911B/OUT
5,FIC_580921,IN,//FI_580921/AI1/OUT
6,PY-580959,INPUT,//PIC_580919/OUT2
7,PY-580959,ALM_HYS,AI1/ALARM_HYS
8,PY-580959,HI_ALM_HYS,AI1/HI_HYS
9,PY-580959,HI_HI_ALM_HYS,AI1/HI_HI_HYS


In [63]:
intit_list = instance_attribute['attr_instance_ref'].dropna().unique().tolist()
intit_list

['//TI_580920/AI1/OUT',
 '//AUX_BMS_IO/PIT911B/OUT',
 '//FI_580931/AI1/OUT',
 '//FI_580921/AI1/OUT',
 '//PIC_580919/OUT2',
 'AI1/ALARM_HYS',
 'AI1/HI_HYS',
 'AI1/HI_HI_HYS',
 'AI1/LO_HYS',
 'AI1/LO_LO_HYS',
 '//AUX_BMS_IO/PIT911B/LSAVTR-HH/BYPASS1',
 '//AUX_BMS_IO/PIT911B/HH_LIM',
 '//AUX_BMS_IO/PIT911B/HI_LIM',
 '//AUX_BMS_IO/PIT911B/LSAVTR-HH/IN_SCALE',
 '//AUX_BMS_IO/PIT911B_ALM/XMTR_FROZEN_ALM']

In [64]:
clean_list = []

for ref in intit_list:
    str_list = ref.split('//')
    str_list = list(filter(None, str_list))
    clean_list.append(str_list)
    
clean_list = [x for y in clean_list for x in y]
clean_list

['TI_580920/AI1/OUT',
 'AUX_BMS_IO/PIT911B/OUT',
 'FI_580931/AI1/OUT',
 'FI_580921/AI1/OUT',
 'PIC_580919/OUT2',
 'AI1/ALARM_HYS',
 'AI1/HI_HYS',
 'AI1/HI_HI_HYS',
 'AI1/LO_HYS',
 'AI1/LO_LO_HYS',
 'AUX_BMS_IO/PIT911B/LSAVTR-HH/BYPASS1',
 'AUX_BMS_IO/PIT911B/HH_LIM',
 'AUX_BMS_IO/PIT911B/HI_LIM',
 'AUX_BMS_IO/PIT911B/LSAVTR-HH/IN_SCALE',
 'AUX_BMS_IO/PIT911B_ALM/XMTR_FROZEN_ALM']

In [65]:
dictionary = {intit_list[i]: clean_list[i] for i in range(len(intit_list))} 
dictionary

{'//TI_580920/AI1/OUT': 'TI_580920/AI1/OUT',
 '//AUX_BMS_IO/PIT911B/OUT': 'AUX_BMS_IO/PIT911B/OUT',
 '//FI_580931/AI1/OUT': 'FI_580931/AI1/OUT',
 '//FI_580921/AI1/OUT': 'FI_580921/AI1/OUT',
 '//PIC_580919/OUT2': 'PIC_580919/OUT2',
 'AI1/ALARM_HYS': 'AI1/ALARM_HYS',
 'AI1/HI_HYS': 'AI1/HI_HYS',
 'AI1/HI_HI_HYS': 'AI1/HI_HI_HYS',
 'AI1/LO_HYS': 'AI1/LO_HYS',
 'AI1/LO_LO_HYS': 'AI1/LO_LO_HYS',
 '//AUX_BMS_IO/PIT911B/LSAVTR-HH/BYPASS1': 'AUX_BMS_IO/PIT911B/LSAVTR-HH/BYPASS1',
 '//AUX_BMS_IO/PIT911B/HH_LIM': 'AUX_BMS_IO/PIT911B/HH_LIM',
 '//AUX_BMS_IO/PIT911B/HI_LIM': 'AUX_BMS_IO/PIT911B/HI_LIM',
 '//AUX_BMS_IO/PIT911B/LSAVTR-HH/IN_SCALE': 'AUX_BMS_IO/PIT911B/LSAVTR-HH/IN_SCALE',
 '//AUX_BMS_IO/PIT911B_ALM/XMTR_FROZEN_ALM': 'AUX_BMS_IO/PIT911B_ALM/XMTR_FROZEN_ALM'}

In [66]:
instance_attribute['attr_instance_ref'] = instance_attribute['attr_instance_ref'].replace(dictionary)
instance_attribute

,module_name,attribute_instance_name,attr_instance_ref
0,FI_580931,TEMP_IN,TI_580920/AI1/OUT
1,FI_580931,PRESS_IN,AUX_BMS_IO/PIT911B/OUT
2,FIC_580931,IN,FI_580931/AI1/OUT
3,FI_580921,TEMP_IN,TI_580920/AI1/OUT
4,FI_580921,PRESS_IN,AUX_BMS_IO/PIT911B/OUT
5,FIC_580921,IN,FI_580921/AI1/OUT
6,PY-580959,INPUT,PIC_580919/OUT2
7,PY-580959,ALM_HYS,AI1/ALARM_HYS
8,PY-580959,HI_ALM_HYS,AI1/HI_HYS
9,PY-580959,HI_HI_ALM_HYS,AI1/HI_HI_HYS


In [67]:
elements = []
for ref in clean_list:
    str_list = ref.split('/')
    elements.append(str_list)
elements

[['TI_580920', 'AI1', 'OUT'],
 ['AUX_BMS_IO', 'PIT911B', 'OUT'],
 ['FI_580931', 'AI1', 'OUT'],
 ['FI_580921', 'AI1', 'OUT'],
 ['PIC_580919', 'OUT2'],
 ['AI1', 'ALARM_HYS'],
 ['AI1', 'HI_HYS'],
 ['AI1', 'HI_HI_HYS'],
 ['AI1', 'LO_HYS'],
 ['AI1', 'LO_LO_HYS'],
 ['AUX_BMS_IO', 'PIT911B', 'LSAVTR-HH', 'BYPASS1'],
 ['AUX_BMS_IO', 'PIT911B', 'HH_LIM'],
 ['AUX_BMS_IO', 'PIT911B', 'HI_LIM'],
 ['AUX_BMS_IO', 'PIT911B', 'LSAVTR-HH', 'IN_SCALE'],
 ['AUX_BMS_IO', 'PIT911B_ALM', 'XMTR_FROZEN_ALM']]

In [68]:
padding_list = []
for element in elements:
    pattern = ['', '', '']
    if len(element) < 3:
        pattern[len(element)-1:] = element
    elif len(element) > 3:
        pattern = element[:-1]
    else:
        pattern[:len(element)] = element
        
    padding_list.append(pattern)
padding_list

[['TI_580920', 'AI1', 'OUT'],
 ['AUX_BMS_IO', 'PIT911B', 'OUT'],
 ['FI_580931', 'AI1', 'OUT'],
 ['FI_580921', 'AI1', 'OUT'],
 ['', 'PIC_580919', 'OUT2'],
 ['', 'AI1', 'ALARM_HYS'],
 ['', 'AI1', 'HI_HYS'],
 ['', 'AI1', 'HI_HI_HYS'],
 ['', 'AI1', 'LO_HYS'],
 ['', 'AI1', 'LO_LO_HYS'],
 ['AUX_BMS_IO', 'PIT911B', 'LSAVTR-HH'],
 ['AUX_BMS_IO', 'PIT911B', 'HH_LIM'],
 ['AUX_BMS_IO', 'PIT911B', 'HI_LIM'],
 ['AUX_BMS_IO', 'PIT911B', 'LSAVTR-HH'],
 ['AUX_BMS_IO', 'PIT911B_ALM', 'XMTR_FROZEN_ALM']]

In [69]:
final_list = []
for i, lis in enumerate(padding_list):
    lis += [clean_list[i]]
    final_list.append(lis)
final_list

[['TI_580920', 'AI1', 'OUT', 'TI_580920/AI1/OUT'],
 ['AUX_BMS_IO', 'PIT911B', 'OUT', 'AUX_BMS_IO/PIT911B/OUT'],
 ['FI_580931', 'AI1', 'OUT', 'FI_580931/AI1/OUT'],
 ['FI_580921', 'AI1', 'OUT', 'FI_580921/AI1/OUT'],
 ['', 'PIC_580919', 'OUT2', 'PIC_580919/OUT2'],
 ['', 'AI1', 'ALARM_HYS', 'AI1/ALARM_HYS'],
 ['', 'AI1', 'HI_HYS', 'AI1/HI_HYS'],
 ['', 'AI1', 'HI_HI_HYS', 'AI1/HI_HI_HYS'],
 ['', 'AI1', 'LO_HYS', 'AI1/LO_HYS'],
 ['', 'AI1', 'LO_LO_HYS', 'AI1/LO_LO_HYS'],
 ['AUX_BMS_IO',
  'PIT911B',
  'LSAVTR-HH',
  'AUX_BMS_IO/PIT911B/LSAVTR-HH/BYPASS1'],
 ['AUX_BMS_IO', 'PIT911B', 'HH_LIM', 'AUX_BMS_IO/PIT911B/HH_LIM'],
 ['AUX_BMS_IO', 'PIT911B', 'HI_LIM', 'AUX_BMS_IO/PIT911B/HI_LIM'],
 ['AUX_BMS_IO',
  'PIT911B',
  'LSAVTR-HH',
  'AUX_BMS_IO/PIT911B/LSAVTR-HH/IN_SCALE'],
 ['AUX_BMS_IO',
  'PIT911B_ALM',
  'XMTR_FROZEN_ALM',
  'AUX_BMS_IO/PIT911B_ALM/XMTR_FROZEN_ALM']]

In [70]:
ref_table = pd.DataFrame(final_list, columns=['ref_module_name', 'ref_function_block', 'ref_attribute', 
                                              'attr_instance_ref'])
ref_table

,ref_module_name,ref_function_block,ref_attribute,attr_instance_ref
0,TI_580920,AI1,OUT,TI_580920/AI1/OUT
1,AUX_BMS_IO,PIT911B,OUT,AUX_BMS_IO/PIT911B/OUT
2,FI_580931,AI1,OUT,FI_580931/AI1/OUT
3,FI_580921,AI1,OUT,FI_580921/AI1/OUT
4,,PIC_580919,OUT2,PIC_580919/OUT2
5,,AI1,ALARM_HYS,AI1/ALARM_HYS
6,,AI1,HI_HYS,AI1/HI_HYS
7,,AI1,HI_HI_HYS,AI1/HI_HI_HYS
8,,AI1,LO_HYS,AI1/LO_HYS
9,,AI1,LO_LO_HYS,AI1/LO_LO_HYS


In [71]:
final_ref_df = instance_attribute.merge(ref_table)
final_ref_df

,module_name,attribute_instance_name,attr_instance_ref,ref_module_name,ref_function_block,ref_attribute
0,FI_580931,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
1,FI_580921,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
2,KS_AI_SIS_HH,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
3,KS_AI_SIS_HH,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
4,PI_580911B,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
5,PI_580911B,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
6,FI_580931,PRESS_IN,AUX_BMS_IO/PIT911B/OUT,AUX_BMS_IO,PIT911B,OUT
7,FI_580921,PRESS_IN,AUX_BMS_IO/PIT911B/OUT,AUX_BMS_IO,PIT911B,OUT
8,KS_AI_SIS_HH,IO_IN,AUX_BMS_IO/PIT911B/OUT,AUX_BMS_IO,PIT911B,OUT
9,KS_AI_SIS_HH,PRESS_IN,AUX_BMS_IO/PIT911B/OUT,AUX_BMS_IO,PIT911B,OUT


In [72]:
final_ref_df.loc[final_ref_df['ref_module_name'] == '', 'ref_module_name'] = final_ref_df['module_name']

In [73]:
final_ref_df

,module_name,attribute_instance_name,attr_instance_ref,ref_module_name,ref_function_block,ref_attribute
0,FI_580931,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
1,FI_580921,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
2,KS_AI_SIS_HH,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
3,KS_AI_SIS_HH,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
4,PI_580911B,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
5,PI_580911B,TEMP_IN,TI_580920/AI1/OUT,TI_580920,AI1,OUT
6,FI_580931,PRESS_IN,AUX_BMS_IO/PIT911B/OUT,AUX_BMS_IO,PIT911B,OUT
7,FI_580921,PRESS_IN,AUX_BMS_IO/PIT911B/OUT,AUX_BMS_IO,PIT911B,OUT
8,KS_AI_SIS_HH,IO_IN,AUX_BMS_IO/PIT911B/OUT,AUX_BMS_IO,PIT911B,OUT
9,KS_AI_SIS_HH,PRESS_IN,AUX_BMS_IO/PIT911B/OUT,AUX_BMS_IO,PIT911B,OUT


# Find wire connections for selected modules

## 1- Define wire_connections extractor function

In [74]:
def wire_connections_variables(module, module_name):

    wire_modul_1 = module.split('WIRE SOURCE')[1:]
    
    if len(wire_modul_1) > 0:
        wire_modul_1 = ['WIRE SOURCE' + wire for wire in  wire_modul_1]
        
        wire_connection_lists = []
        
        for conn in wire_modul_1:

            # source
            regex=re.compile(r'SOURCE=.* D')
            source = regex.findall(conn)[0][8:-3]

            # destination
            regex=re.compile(r'DESTINATION=.*')
            destination = regex.findall(conn)[0][13:-1]
            
            wire_connection_lists.append([module_name, source, destination])

        return wire_connection_lists
    else:
        return [[module_name, np.nan, np.nan]]

## 2- Find wire connections for each selected module

In [75]:
modules = data.split('\nMODULE_CLASS')[1:]
modules = ['MODULE_CLASS' + module for module in modules]

wire_connection_list = []
for module_id in selected_modules_id:
    wire_connection_list.append(wire_connections_variables(all_actual_modules[module_id], 
                                            all_modules.iloc[module_id]['module_name']))
    
wire_connection_list = [item for sublist in wire_connection_list for item in sublist]

In [76]:
wire_connections = pd.DataFrame(wire_connection_list, columns=['module_name', 'source', 'destination']).dropna()

wire_connections

,module_name,source,destination
0,FI_580931,HI_HI_ALARM/OUT_D,AI1/HI_HI_ENABLE
1,FI_580931,HI_ALARM/OUT_D,AI1/HI_ENABLE
2,FI_580931,LO_ALARM/OUT_D,AI1/LO_ENABLE
3,FI_580931,LO_LO_ALARM/OUT_D,AI1/LO_LO_ENABLE
4,FI_580931,AI2/OUT,AGA_US1/IN
...,...,...,...
377,PI_580911B,ALM_HYS,LO_LO_ALM_HYS
378,PI_580911B,HI_HI_ALARM/OUT_D,AI1/HI_HI_ENAB
379,PI_580911B,HI_ALARM/OUT_D,AI1/HI_ENAB
380,PI_580911B,LO_ALARM/OUT_D,AI1/LO_ENAB


In [77]:
# index_list_dest = wire_connections['destination'].str.find('/').values
# index_list_dest = [int(x) for x in index_list_dest]


# index_list_source = wire_connections['source'].str.find('/').values
# index_list_source = [int(x) for x in index_list_source]

# for i in range(len(index_list_dest)):
#     wire_connections['destination'].iloc[i] = wire_connections['destination'].iloc[i][:index_list_dest[i]]
#     wire_connections['source'].iloc[i] = wire_connections['source'].iloc[i][:index_list_source[i]]
# wire_connections

## 1- source distribution and replacement

In [78]:
correct_source = wire_connections[wire_connections['source'].str.find('/') != -1.0]
correct_source

,module_name,source,destination
0,FI_580931,HI_HI_ALARM/OUT_D,AI1/HI_HI_ENABLE
1,FI_580931,HI_ALARM/OUT_D,AI1/HI_ENABLE
2,FI_580931,LO_ALARM/OUT_D,AI1/LO_ENABLE
3,FI_580931,LO_LO_ALARM/OUT_D,AI1/LO_LO_ENABLE
4,FI_580931,AI2/OUT,AGA_US1/IN
...,...,...,...
373,PI_580911B,BG1/BKCAL_OUT,CALC1/IN2
378,PI_580911B,HI_HI_ALARM/OUT_D,AI1/HI_HI_ENAB
379,PI_580911B,HI_ALARM/OUT_D,AI1/HI_ENAB
380,PI_580911B,LO_ALARM/OUT_D,AI1/LO_ENAB


In [79]:
instance_source = wire_connections[wire_connections['source'].str.find('/') == -1]
instance_source = instance_source.rename(columns={'source':'attribute_instance_name'}).reset_index(drop=True)
instance_source

,module_name,attribute_instance_name,destination
0,FI_580931,PRESS_IN,ADD1/IN1
1,FI_580931,TEMP_IN,AGA_US1/TEMP_IN
2,FI_580931,P_ATM,ADD1/IN2
3,FIC_580931,IN,PV_CALC/FUEL_FLOW
4,FIC_580931,REMOTE_SP,PID1/CAS_IN
...,...,...,...
81,PI_580911B,TUNING_MODE,CENTER_GAS/TUNING_MODE
82,PI_580911B,ALM_HYS,HI_HI_ALM_HYS
83,PI_580911B,ALM_HYS,HI_ALM_HYS
84,PI_580911B,ALM_HYS,LO_ALM_HYS


In [80]:
source_tmp = instance_source.merge(final_ref_df)
source_tmp = source_tmp[['module_name', 'ref_function_block', 'destination', 'ref_module_name']].rename(
                columns={'ref_function_block':'source',
                        'ref_module_name':'source_fb_name'}).reset_index(drop=True)
source_tmp

,module_name,source,destination,source_fb_name
0,FI_580931,PIT911B,ADD1/IN1,AUX_BMS_IO
1,FI_580931,AI1,AGA_US1/TEMP_IN,TI_580920
2,FIC_580931,AI1,PV_CALC/FUEL_FLOW,FI_580931
3,FIC_580931,AI1,PV_EU,FI_580931
4,FI_580921,PIT911B,ADD1/IN1,AUX_BMS_IO
...,...,...,...,...
56,PI_580911B,PIC_580919,BG1/IN_1,PI_580911B
57,PI_580911B,AI1,HI_HI_ALM_HYS,PI_580911B
58,PI_580911B,AI1,HI_ALM_HYS,PI_580911B
59,PI_580911B,AI1,LO_ALM_HYS,PI_580911B


In [81]:
final_source = source_tmp.append(correct_source)
final_source

,destination,module_name,source,source_fb_name
0,ADD1/IN1,FI_580931,PIT911B,AUX_BMS_IO
1,AGA_US1/TEMP_IN,FI_580931,AI1,TI_580920
2,PV_CALC/FUEL_FLOW,FIC_580931,AI1,FI_580931
3,PV_EU,FIC_580931,AI1,FI_580931
4,ADD1/IN1,FI_580921,PIT911B,AUX_BMS_IO
...,...,...,...,...
373,CALC1/IN2,PI_580911B,BG1/BKCAL_OUT,NaN
378,AI1/HI_HI_ENAB,PI_580911B,HI_HI_ALARM/OUT_D,NaN
379,AI1/HI_ENAB,PI_580911B,HI_ALARM/OUT_D,NaN
380,AI1/LO_ENAB,PI_580911B,LO_ALARM/OUT_D,NaN


## 1- destination distribution and replacement

In [82]:
correct_dest = final_source[final_source['destination'].str.find('/') != -1.0]
correct_dest

,destination,module_name,source,source_fb_name
0,ADD1/IN1,FI_580931,PIT911B,AUX_BMS_IO
1,AGA_US1/TEMP_IN,FI_580931,AI1,TI_580920
2,PV_CALC/FUEL_FLOW,FIC_580931,AI1,FI_580931
4,ADD1/IN1,FI_580921,PIT911B,AUX_BMS_IO
5,AGA_US1/TEMP_IN,FI_580921,AI1,TI_580920
...,...,...,...,...
373,CALC1/IN2,PI_580911B,BG1/BKCAL_OUT,NaN
378,AI1/HI_HI_ENAB,PI_580911B,HI_HI_ALARM/OUT_D,NaN
379,AI1/HI_ENAB,PI_580911B,HI_ALARM/OUT_D,NaN
380,AI1/LO_ENAB,PI_580911B,LO_ALARM/OUT_D,NaN


In [83]:
instance_destination = final_source[final_source['destination'].str.find('/') == -1]
instance_destination = instance_destination.rename(
                columns={'destination':'attribute_instance_name'}).reset_index(drop=True)
instance_destination## 1- source distribution and replacement

,attribute_instance_name,module_name,source,source_fb_name
0,PV_EU,FIC_580931,AI1,FI_580931
1,PV_EU,FIC_580921,AI1,FI_580921
2,HI_HI_ALM_HYS,PY-580959,AI1,PY-580959
3,HI_ALM_HYS,PY-580959,AI1,PY-580959
4,LO_ALM_HYS,PY-580959,AI1,PY-580959
5,LO_LO_ALM_HYS,PY-580959,AI1,PY-580959
6,BYPASS_ACTIVE,KS_AI_SIS_HH,PIT911B,AUX_BMS_IO
7,PV_EU,KS_AI_SIS_HH,AI1,FI_580931
8,PV_EU,KS_AI_SIS_HH,AI1,FI_580921
9,PV_EU,KS_AI_SIS_HH,AI1,FI_580931


In [84]:
dest_temp = instance_destination.merge(final_ref_df)

dest_temp = dest_temp[['module_name', 'source', 'ref_function_block','ref_module_name', 
                'source_fb_name']].rename(columns={'ref_function_block':'destination',
                        'ref_module_name':'dest_fb_name'}).reset_index(drop=True)
dest_temp

,module_name,source,destination,dest_fb_name,source_fb_name
0,PY-580959,AI1,AI1,PY-580959,PY-580959
1,PY-580959,AI1,AI1,PY-580959,PY-580959
2,PY-580959,AI1,AI1,PY-580959,PY-580959
3,PY-580959,AI1,AI1,PY-580959,PY-580959
4,KS_AI_STD,AI1,AI1,KS_AI_STD,KS_AI_STD
5,KS_AI_STD,AI1,AI1,KS_AI_STD,KS_AI_STD
6,KS_AI_STD,AI1,AI1,KS_AI_STD,KS_AI_STD
7,KS_AI_STD,AI1,AI1,KS_AI_STD,KS_AI_STD
8,PI_580911B,AI1,AI1,PI_580911B,PI_580911B
9,PI_580911B,AI1,AI1,PI_580911B,PI_580911B


In [92]:
final_dest = dest_temp.append(correct_dest)
final_dest = final_dest.reset_index(drop=True)
final_dest

,dest_fb_name,destination,module_name,source,source_fb_name
0,PY-580959,AI1,PY-580959,AI1,PY-580959
1,PY-580959,AI1,PY-580959,AI1,PY-580959
2,PY-580959,AI1,PY-580959,AI1,PY-580959
3,PY-580959,AI1,PY-580959,AI1,PY-580959
4,KS_AI_STD,AI1,KS_AI_STD,AI1,KS_AI_STD
...,...,...,...,...,...
316,NaN,CALC1/IN2,PI_580911B,BG1/BKCAL_OUT,NaN
317,NaN,AI1/HI_HI_ENAB,PI_580911B,HI_HI_ALARM/OUT_D,NaN
318,NaN,AI1/HI_ENAB,PI_580911B,HI_ALARM/OUT_D,NaN
319,NaN,AI1/LO_ENAB,PI_580911B,LO_ALARM/OUT_D,NaN


In [93]:
# final_dest['destination'].str.find('AI1')

index_list_dest = final_dest['destination'].str.find('/').values
index_list_dest = [int(x) for x in index_list_dest]


index_list_source = final_dest['source'].str.find('/').values
index_list_source = [int(x) for x in index_list_source]

for i in range(len(index_list_dest)):
    final_dest['destination'].iloc[i] = final_dest['destination'].iloc[i][:index_list_dest[i]]
    
for i in range(len(index_list_source)):
    final_dest['source'].iloc[i] = final_dest['source'].iloc[i][:index_list_source[i]]
final_dest

,dest_fb_name,destination,module_name,source,source_fb_name
0,PY-580959,AI,PY-580959,AI,PY-580959
1,PY-580959,AI,PY-580959,AI,PY-580959
2,PY-580959,AI,PY-580959,AI,PY-580959
3,PY-580959,AI,PY-580959,AI,PY-580959
4,KS_AI_STD,AI,KS_AI_STD,AI,KS_AI_STD
...,...,...,...,...,...
316,NaN,CALC1,PI_580911B,BG1,NaN
317,NaN,AI1,PI_580911B,HI_HI_ALARM,NaN
318,NaN,AI1,PI_580911B,HI_ALARM,NaN
319,NaN,AI1,PI_580911B,LO_ALARM,NaN


In [94]:
final_dest.loc[final_dest['dest_fb_name'].isna(), 'dest_fb_name'] = final_dest['module_name']
final_dest.loc[final_dest['source_fb_name'].isna(), 'source_fb_name'] = final_dest['module_name']

In [100]:
final_df = final_dest.drop('module_name',1).drop_duplicates().reset_index(drop=True)
final_df

,dest_fb_name,destination,source,source_fb_name
0,PY-580959,AI,AI,PY-580959
1,KS_AI_STD,AI,AI,KS_AI_STD
2,PI_580911B,AI,AI,PI_580911B
3,FI_580931,ADD1,PIT911,AUX_BMS_IO
4,FI_580931,AGA_US1,AI,TI_580920
...,...,...,...,...
221,PI_580911B,ACT2,PDE2,PI_580911B
222,PI_580911B,RTLM2,BG1,PI_580911B
223,PI_580911B,RTLM1,RTLM2,PI_580911B
224,PI_580911B,RTLM2,CND4,PI_580911B


In [101]:
final_df[['source', 'destination']].to_csv('graph.csv', index=False)